In [734]:
import numpy as np
from numpy import random
import copy

Simulates the remainder of the regular season. Produces a dictionary of sorted regular season standings

In [735]:
previous_standings = {"Brewers": [95,62, 1, "Central"], "Phillies": [92,64, 2, "East"], 
                     "Dodgers": [88,68, 3, "West"], "Cubs": [88,68, 4, "Central"], 
                     "Padres": [86, 71, 5, "West"], "Mets": [80, 76, 6, "East"], 
                     "Reds": [80, 76, 7, "Central"], "Diamondbacks": [79, 77, 8, "West"]}

def simRegularSeason(current_standings):
    new_standings = copy.deepcopy(current_standings)
    # Simulate Dodgers & Diamondbacks
    dodgers_wins = np.sum(np.random.binomial(3, .5))
    diamondbacks_wins = 3 - dodgers_wins
    dodgers_wins += np.sum(np.random.binomial(3, .5))
    diamondbacks_wins += np.sum(np.random.binomial(3, .5))
    new_standings["Dodgers"][0] += dodgers_wins
    new_standings["Dodgers"][1] = 162 - new_standings["Dodgers"][0]
    new_standings["Diamondbacks"][0] += diamondbacks_wins
    new_standings["Diamondbacks"][1] = 162 - new_standings["Diamondbacks"][0] 

    # Simulate Cubs and Mets
    cubs_wins = np.sum(np.random.binomial(3, .5))
    mets_wins = 3 - cubs_wins
    cubs_wins += np.sum(np.random.binomial(3, .5))
    mets_wins += np.sum(np.random.binomial(3, .5))
    new_standings["Cubs"][0] += cubs_wins
    new_standings["Cubs"][1] = 162 - new_standings["Cubs"][0]
    new_standings["Mets"][0] += mets_wins
    new_standings["Mets"][1] = 162 - new_standings["Mets"][0] 

    # Simulate Brewers and Padres
    brewers_wins = np.sum(np.random.binomial(2, .5))
    padres_wins = 2 - brewers_wins
    brewers_wins += np.sum(np.random.binomial(3, .5))
    padres_wins += np.sum(np.random.binomial(3, .5))
    new_standings["Brewers"][0] += brewers_wins
    new_standings["Brewers"][1] = 162 - new_standings["Brewers"][0]
    new_standings["Padres"][0] += padres_wins
    new_standings["Padres"][1] = 162 - new_standings["Padres"][0] 

    # Simulate Reds and Phillies
    reds_wins = np.sum(np.random.binomial(6, .5))
    phillies_wins = np.sum(np.random.binomial(6, .5))
    new_standings["Reds"][0] += reds_wins
    new_standings["Reds"][1] = 162 - new_standings["Reds"][0]
    new_standings["Phillies"][0] += phillies_wins
    new_standings["Phillies"][1] = 162 - new_standings["Phillies"][0] 
    return dict(sorted(new_standings.items(), key = lambda item : (item[1][1], item[1][2])))


Given regular season standings, finds the division leaders and sorts them by wins and then by the tiebreaker if necessary. Produces a dictionary of sorted division leaders

In [736]:
def div_leaders(regular_season_standings):
    central_standings = dict(filter(lambda item: item[1][3] == "Central", regular_season_standings.items()))
    east_standings = dict(filter(lambda item: item[1][3] == "East", regular_season_standings.items()))
    west_standings = dict(filter(lambda item: item[1][3] == "West", regular_season_standings.items()))
    central_top = dict([next(iter(central_standings.items()))])
    east_top = dict([next(iter(east_standings.items()))])
    west_top = dict([next(iter(west_standings.items()))])

    combined = central_top | east_top | west_top
    return dict(sorted(combined.items(), key = lambda item : (item[1][1], item[1][2])))


Combines division leaders with other teams to create a dictionary of the final seeding

In [737]:
def get_final_standings(div_leaders, full_standings):
    full_standings = dict(filter(lambda item: item[0] not in div_leaders.keys(), full_standings.items()))
    final_standings = div_leaders | full_standings
    return final_standings


Simulates the 3-6 wildcard or 4-5 wildcard depending on the Brewer's postseason seed. Returns whether the Brewers will play the Padres

In [738]:
def find_brewers_opponent(postseason_seeding):
    brewers_seed = list(postseason_seeding.keys()).index('Brewers') + 1

    if (brewers_seed == 1):
        four_seed = list(postseason_seeding.items())[3][0]
        five_seed = list(postseason_seeding.items())[4][0]
        four_seed_wins = np.sum(np.random.binomial(3, .55))
        return four_seed == 'Padres' if four_seed_wins > 1 else five_seed == 'Padres'
    
    elif (brewers_seed == 2):
        three_seed = list(postseason_seeding.items())[2][0]
        six_seed = list(postseason_seeding.items())[5][0]
        three_seed_wins = np.sum(np.random.binomial(3, .55))
        return three_seed == 'Padres' if three_seed_wins > 1 else six_seed == 'Padres'

Simulates the process 100,000 times to estimate the probability that the Brewers face the Padres in the divisional series

In [742]:
def find_prob_face_pads():
    outcomes = []
    for _ in range(100000):
        standings = simRegularSeason(previous_standings)
        division_leaders = div_leaders(standings)
        seeding = get_final_standings(division_leaders, standings)
        outcomes.append(find_brewers_opponent(seeding))
    
    return np.mean(outcomes)

find_prob_face_pads()

np.float64(0.40574)

Based on 10000 simulations, the Brewers have a 40.57% chance to face the Padres in the divisional series